In [1]:
import h5py
import numpy as np
import shutil

/home/hatta/.virtualenvs/Singleshot/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
weights_source_path = 'trained_weights/VGG_coco_SSD_300x300_iter_400000.h5'
weights_destination_path = 'trained_weights/VGG_coco_SSD_300x300_iter_400000_subsampled.h5'
shutil.copy(weights_source_path, weights_destination_path)
weights_source_file = h5py.File(weights_source_path, 'r')
weights_destination_file = h5py.File(weights_destination_path)

In [3]:
from tensor_sampling_utils import sample_tensors
classifier_names = ['conv4_3_norm_mbox_conf',
                    'fc7_mbox_conf',
                    'conv6_2_mbox_conf',
                    'conv7_2_mbox_conf',
                    'conv8_2_mbox_conf',
                    'conv9_2_mbox_conf']
n_classes_source = 81
# ['background', 'car', 'truck', 'pedestrian', stop_sign']
classes_of_interest = [0, 3, 8, 1, 12]

for name in classifier_names:
    kernel = weights_source_file[name][name]['kernel:0'].value
    bias = weights_source_file[name][name]['bias:0'].value
    height, width, in_channels, out_channels = kernel.shape
    subsampling_indices = []
    for i in range(int(out_channels/n_classes_source)):
        indices = np.array(classes_of_interest) + i * n_classes_source
        subsampling_indices.append(indices)
    subsampling_indices = list(np.concatenate(subsampling_indices))
    new_kernel, new_bias = sample_tensors(weights_list=[kernel, bias],
                                          sampling_instructions=[height, width, in_channels, subsampling_indices],
                                          axes=[[3]], # The one bias dimension corresponds to the last kernel dimension.
                                          init=['gaussian', 'zeros'],
                                          mean=0.0,
                                          stddev=0.005)
    del weights_destination_file[name][name]['kernel:0']
    del weights_destination_file[name][name]['bias:0']
    weights_destination_file[name][name].create_dataset(name='kernel:0', data=new_kernel)
    weights_destination_file[name][name].create_dataset(name='bias:0', data=new_bias)

ModuleNotFoundError: No module named 'tensor_sampling_utils'

In [15]:
for name in classifier_names:
    res_kernel = weights_destination_file[name][name]['kernel:0']
    res_bias = weights_destination_file[name][name]['bias:0']

    print("Shape of the '{}' weights:".format(name))
    print("kernel:\t", res_kernel.shape)
    print("bias:\t", res_bias.shape)
    print()

Shape of the 'conv4_3_norm_mbox_conf' weights:
kernel:	 (3, 3, 512, 20)
bias:	 (20,)

Shape of the 'fc7_mbox_conf' weights:
kernel:	 (3, 3, 1024, 30)
bias:	 (30,)

Shape of the 'conv6_2_mbox_conf' weights:
kernel:	 (3, 3, 512, 30)
bias:	 (30,)

Shape of the 'conv7_2_mbox_conf' weights:
kernel:	 (3, 3, 256, 30)
bias:	 (30,)

Shape of the 'conv8_2_mbox_conf' weights:
kernel:	 (3, 3, 256, 20)
bias:	 (20,)

Shape of the 'conv9_2_mbox_conf' weights:
kernel:	 (3, 3, 256, 20)
bias:	 (20,)

